### 4장 1번
* 수백만 개의 특성을 가진 훈련세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?

-> 확률적 경사 하강법을 사용하는 것이 적합하다. 그 이유는 한번에 전체 데이터를 사용하지 않고 한 개의 샘플을 랜덤으로 선택하고 그 하나의 샘플에 대한 가중치를 업데이트하는 방식이기에 수백만 개의 특성을 가진(큰 데이터셋을 가진) 훈련세트에서도 메모리 효율적으로 학습이 가능하기 때문이다.


## 4장 2번

* 훈련 세트에 있는 특성들이 서로 다른 스케일을 가지고 있습니다. 이런 데이터에 잘 작동하지 않는 알고리즘은? 이 문제는 어떻게 해결해야할까?

-> 스케일이 다르다는 것은 특성들의 값의 범위(단위나 크기 등)의 차이를 의미한다.

-> 거리 기반으로 하는 선형 회귀, 로지스틱 회귀 등이 영향을 많이 받는다. 표준화를 사용하여 해결할 수 있다.



## 4장 3번

* 경사 하강법으로 로지스틱 회귀 모델을 훈련시킬 때 지역 최솟값에 갇힐 가능성이 있을까요?

-> 로지스틱 회귀 모델의 비용함수는 볼록 함수이므로 경사 하강법이 훈련될 때 지역 최솟값에 갇힐 가능성이 없다.

## 4장 12번
사이킷런을 사용하지 않고 넘파이만 사용하여 조기 종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀를 구현해보세요. 이를 붓꽃 데이터셋 같은 분류 작업에 사용해보세요.

### Softmax Regression (소프트맥스 회귀)란?
다중 클래스 분류 문제(예: 붓꽃 데이터셋)에서 사용되는 확률 기반 모델.

입력 데이터가 주어졌을 때, 각 클래스(예: Setosa, Versicolor, Virginica)에 속할 확률을 출력.



데이터셋의 y값이 클래스 인덱스 (0,1,2)로 되어 있어서 소프트맥스 회귀 모델을 학습하기 위해 원-핫 벡터 형식으로 변환해야함.

In [ ]:
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris(as_frame=True)

# DataFrame에서 꽃잎 길이와 너비 선택 후 NumPy 배열로 변환
X = iris["data"].iloc[:, [2, 3]].values
y = iris["target"].values  # NumPy 배열로 변환

# 편향 항 추가 -> 1로 채워진 열벡터를 생성(첫 번째 열이 항상 1인 편향 항)
X_with_bias = np.c_[np.ones([len(X), 1]), X]

# 난수 시드 설정
np.random.seed(2042)

In [ ]:
test_ratio = 0.2          # 테스트 세트 비율 (20%)
validation_ratio = 0.2     # 검증 세트 비율 (20%)
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)  # 테스트 데이터 개수
validation_size = int(total_size * validation_ratio)  # 검증 데이터 개수
train_size = total_size - test_size - validation_size  # 훈련 데이터 개수

In [ ]:
rnd_indices = np.random.permutation(total_size)  # NumPy 배열 유지

# 데이터셋 분할
X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]

X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]

X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]


In [ ]:

def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [ ]:

y_train[:10]

In [ ]:

to_one_hot(y_train[:10])

In [ ]:

Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

In [ ]:

def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [ ]:
n_inputs = X_train.shape[1] # == 3 (2 features plus the bias term)
n_outputs = len(np.unique(y_train))   # == 3 (3 iris classes)


eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    if iteration % 500 == 0:
        loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients


In [ ]:
Theta

In [ ]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

In [ ]:

eta = 0.1
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1  # 규제 하이퍼파라미터

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    if iteration % 500 == 0:
        xentropy_loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
        loss = xentropy_loss + alpha * l2_loss
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

In [ ]:

logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

In [ ]:
eta = 0.1
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1  # 규제 하이퍼파라미터
best_loss = np.inf

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

    logits = X_valid.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(Y_valid_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    if iteration % 500 == 0:
        print(iteration, loss)
    if loss < best_loss:
        best_loss = loss
    else:
        print(iteration - 1, best_loss)
        print(iteration, loss, "early stopping!")
        break

In [ ]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

In [ ]:
## 다른 방법으로 코드 구현

import numpy as np
from sklearn.datasets import load_iris

# 붓꽃 데이터 로드 및 전처리
iris = load_iris()
X = iris["data"][:, (2, 3)]  # 꽃잎 길이(petal length)와 너비(petal width)만 사용
y = iris["target"]  # 클래스 레이블 (0: Setosa, 1: Versicolor, 2: Virginica)

# 편향 항 추가 (Bias term, x0 = 1 추가)
X_with_bias = np.c_[np.ones([len(X), 1]), X]

# 원-핫 인코딩 (One-hot encoding)
n_classes = np.max(y) + 1  # 3개 클래스 (0, 1, 2)
y_one_hot = np.eye(n_classes)[y]  # 각 클래스를 원-핫 벡터로 변환

# 데이터 분할 (훈련: 60%, 검증: 20%, 테스트: 20%)
np.random.seed(2042)
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)  # 데이터 섞기
X_train = X_with_bias[rnd_indices[:train_size]]
y_train_one_hot = y_one_hot[rnd_indices[:train_size]]

X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid_one_hot = y_one_hot[rnd_indices[train_size:-test_size]]

X_test = X_with_bias[rnd_indices[-test_size:]]
y_test_one_hot = y_one_hot[rnd_indices[-test_size:]]

# 소프트맥스 함수 구현
def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # 오버플로우 방지
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

# 학습 설정
n_inputs = X_train.shape[1]  # 입력 특성 개수 (2 + 1 = 3)
n_outputs = n_classes  # 출력 클래스 개수 (3)

# 가중치 초기화
Theta = np.random.randn(n_inputs, n_outputs)

# 학습 하이퍼파라미터
eta = 0.1  # 학습률
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7  # log 계산 시 0 방지
alpha = 0.1  # L2 정규화 하이퍼파라미터
best_loss = np.inf  # 최적 손실값

# 배치 경사 하강법 (Batch Gradient Descent) + 조기 종료
for iteration in range(n_iterations):
    logits = X_train.dot(Theta)  # 선형 회귀 계산
    Y_proba = softmax(logits)  # 소프트맥스 적용
    error = Y_proba - y_train_one_hot  # 예측값 - 실제값

    # 그래디언트 계산 (L2 정규화 포함)
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]

    # 가중치 업데이트
    Theta = Theta - eta * gradients

    # 검증 세트에서 손실 계산
    logits_valid = X_valid.dot(Theta)
    Y_proba_valid = softmax(logits_valid)
    xentropy_loss = -np.mean(np.sum(y_valid_one_hot * np.log(Y_proba_valid + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss  # 총 손실

    if iteration % 500 == 0:
        print(f"Iteration {iteration}: Loss = {loss:.5f}")

    # 조기 종료 (Early Stopping)
    if loss < best_loss:
        best_loss = loss  # 최적 손실 업데이트
    else:
        print(f"Iteration {iteration-1}: Best Loss = {best_loss:.5f}")
        print(f"Iteration {iteration}: Loss = {loss:.5f} (Early Stopping!)")
        break  # 손실이 증가하면 학습 종료

# 최종 모델 평가
logits_test = X_test.dot(Theta)
Y_proba_test = softmax(logits_test)
y_pred = np.argmax(Y_proba_test, axis=1)  # 확률이 가장 높은 클래스 선택
y_test_labels = np.argmax(y_test_one_hot, axis=1)  # 실제 정답

accuracy = np.mean(y_pred == y_test_labels)  # 정확도 계산
print(f"\n테스트 정확도: {accuracy:.4f}")
